In [1]:
import pandas as pd

In [3]:
WORKDIR="/workdir/dwm269/scMuscle2"
os.chdir(WORKDIR)

In [4]:
meta = pd.read_csv("scMuscle2_metadata_v1-0.csv")
meta = meta.loc[meta["include"],] # remove unwanted metadata
# meta = meta[[x in ["fastq", "bam"] for x in meta["file.format"]]] # remove samples w/ download issues

# meta = meta[[x in ["muscle", "tendon"] for x in meta["tissue"]]] # subset by tissue 
# meta = meta[[x in ["Homo sapiens"] for x in meta["species"]]] # subset by species , "Mus musculus"
# meta = meta.iloc[[8,12,23,34,45],] # subset by row index

meta = meta[[x != "" for x in meta["GSM.accession"]]] # 
meta = meta[[x != "" for x in meta["sample"]]] # 

meta.index = list(range(0, len(meta))) # reset row indices

Quick way to gather DOIs, given pubmed IDs/links

In [6]:
import requests
from bs4 import BeautifulSoup

# pubmed_url = f"https://pubmed.ncbi.nlm.nih.gov/{PMID}/"
def get_doi_from_pubmed(pubmed_url):
    response = requests.get(pubmed_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        doi_element = soup.find('a', {'data-ga-action': 'DOI'})

        if doi_element:
            return doi_element.text
        else:
            print("DOI not found.")
    else:
        print("Error fetching the PubMed page.")
        return None

In [ ]:
pubmed_url = "https://pubmed.ncbi.nlm.nih.gov/30760602/"
doi = get_doi_from_pubmed(pubmed_url)

if doi:
    print("DOI:", doi)

In [12]:
doi_list = []
for link in meta["manuscript.pubmed"]:
    if str(link).startswith('http'):
        doi = get_doi_from_pubmed(link)
        if doi:
            doi_list.append(doi)
        else:
            doi_list.append("Not found")
    else:
        # print("Invalid URL encountered:", link)
        doi_list.append("Not found")


DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.
DOI not found.
Error fetching the PubMed page.


In [15]:
print("DOIs retrieved:")
# for doi in doi_list:
for i in list(range(0,len(doi_list))):
    if("/" in doi):
        print(f"{meta['manuscript.pubmed'][i]} ||| {doi_list[i]}")

DOIs retrieved:
https://pubmed.ncbi.nlm.nih.gov/30283141/ ||| 
      10.1038/s41586-018-0590-4
    
https://pubmed.ncbi.nlm.nih.gov/30283141/ ||| 
      10.1038/s41586-018-0590-4
    
https://pubmed.ncbi.nlm.nih.gov/30283141/ ||| 
      10.1038/s41586-018-0590-4
    
https://pubmed.ncbi.nlm.nih.gov/30283141/ ||| 
      10.1038/s41586-018-0590-4
    
https://pubmed.ncbi.nlm.nih.gov/30283141/ ||| 
      10.1038/s41586-018-0590-4
    
https://pubmed.ncbi.nlm.nih.gov/30283141/ ||| 
      10.1038/s41586-018-0590-4
    
https://pubmed.ncbi.nlm.nih.gov/31809738/ ||| 
      10.1016/j.stem.2019.11.004
    
https://pubmed.ncbi.nlm.nih.gov/31809738/ ||| 
      10.1016/j.stem.2019.11.004
    
https://pubmed.ncbi.nlm.nih.gov/31809738/ ||| 
      10.1016/j.stem.2019.11.004
    
https://pubmed.ncbi.nlm.nih.gov/31809738/ ||| 
      10.1016/j.stem.2019.11.004
    
https://pubmed.ncbi.nlm.nih.gov/31809738/ ||| 
      10.1016/j.stem.2019.11.004
    
https://pubmed.ncbi.nlm.nih.gov/30922843/ ||| 
      10